In [6]:
using Pkg; Pkg.activate(".")

using Revise
using Dates

using CSV
using DataFrames
using DelimitedFiles
using Printf


  Activating project at `~/Projects/lidar/lidar2019`


In [ ]:

function read_streamlinexr(file::String, nrange::Int)
    # Read data from file skipping header
    x = CSV.read(file, DataFrames.header=false, DataFrame, limit=10000, footerskip=4)

    # get ray time and angles
    # Decimal time (hours)  Azimuth (degrees)  Elevation (degrees) Pitch (degrees) Roll (degrees)
    timeangles = x[1:nrange:end, :]
    a = Dict(
        :dechour => timeangles[!, 1],
        :azimuth => timeangles[!, 2],
        :elevation => timeangles[!, 3],
        :pitch => timeangles[!, 4],
        :roll => timeangles[!, 5]
    )

    nt = length(a[:dechour])

    # get ray data
    idata = mod.(0:size(x, 1)-1, nrange+1) .> 0
    a[:rangegate ] = reshape(x[idata, 1], (nrange, nt))
    a[:dopplervel] = reshape(x[idata, 2], (nrange, nt))
    a[:intensity ] = reshape(x[idata, 3], (nrange, nt))
    a[:beta      ] = reshape(x[idata, 4], (nrange, nt))

    return a
end


In [20]:
# using DelimitedFiles
# using Printf

file_path = "./data/Stare_06_20190710_06.hpl"

function read_streamlinexr(file_path)
    nlines = countlines(file_path)

    # open and read the file
    open(file_path) do file
        nlines = countlines(file_path)

        for _ in 1:2 # skip lines
            readline(file)
        end

        # read number of gates from line 3
        ngates = parse(Int32, last(split(readline(file)))) # 104
        # read gate length from line 4
        gatelength = parse(Float64, last(split(readline(file))))
        for _ in 5:6 # skip
            readline(file)
        end

        # read number of rays from line 7
        nrays = parse(Int32, last(split(readline(file)))) # 1 for Stare or 40 for User
        for _ in 8:9 # skip
            readline(file)
        end

        # read start time from line 10
        tl = split(readline(file))
        start_time = DateTime(tl[end-1]*tl[end], "yyyymmddHH:MM:SS.ss")
        # read velocity resolution (m/s) line 11
        vel_resolution = parse(Float64, last(split(readline(file))))
        readline(file) # skip line 12
        # read header 1 from line 13
        hdrln = split(readline(file))[5:end]
        headr1 = hdrln[1:2:end]
        units1 = hdrln[2:2:end]
        readline(file) # skip line 14
        # read header 2 from line 15
        hdrln = split(readline(file))[4:end] # but don't parse it
        headr2 = ["Range Gate", "Doppler", "Intensity", "Beta"]
        units2 = ["none", "(m/s)", "(SNR + 1)", "(m-1 sr-1)"]
        for _ in 16:17 # skip
            readline(file)
        end

        # now DATA
        # beams could be rays or times
        nbeams = round(Int, (nlines-17) / (1+ngates)) # = nrays*ntimes
        data1 = zeros(nbeams, 5)
        data2 = zeros(nbeams, ngates, 4)
        # for User wind profiles beam <--> VAD ray
        # for Stare beam <--> time
        for ibeam = 1:nbeams
            # beam described by a batch of 1+ngates lines
            # Read the beam parameter line
            line = readline(file)
            data1[ibeam,:] .= parse.(Float64, split(line))
            # Read each gate in the beam
            for igate = 1:ngates
                line = readline(file)
                data2[ibeam, igate,:] .= parse.(Float64, split(line))
            end
        end
    end # close the file

    # parse the variables
    # beam
    time      = data1[:,1] # decimal hours
    azimuth   = data1[:,2] # degrees
    elevangle = data1[:,3] # degrees
    pitch     = data1[:,4]
    roll      = data1[:,5]
    # gate
    height = (data2[1,:,1].+0.5) .* gatelength # center of gate

    # dependent variables (beam, gate)
    dopplervel = data2[:,:,2] # m/s
    intensity  = data2[:,:,3] # SNR + 1
    beta       = data2[:,:,4] # m-1 sr-1  backscatter?

    return time, azimuth, elevangle, pitch, roll, height, dopplervel, intensity, beta
end

read_streamlinexr (generic function with 1 method)

In [19]:
file_path = "./data/Stare_06_20190710_06.hpl"

nlines = countlines(file_path)

file = open(file_path)
for _ in 1:2 # skip lines
    readline(file)
end

# read number of gates from line 3
ngates = parse(Int32, last(split(readline(file)))) # 104
# read gate length from line 4
gatelength = parse(Float64, last(split(readline(file))))
for _ in 5:6 # skip
    readline(file)
end

# read number of rays from line 7
nrays = parse(Int32, last(split(readline(file)))) # 1 for Stare or 40 for User
for _ in 8:9 # skip
    readline(file)
end

# read start time from line 10
tl = split(readline(file))
start_time = DateTime(tl[end-1]*tl[end], "yyyymmddHH:MM:SS.ss")
# read velocity resolution (m/s) line 11
vel_resolution = parse(Float64, last(split(readline(file))))
readline(file) # skip line 12
# read header 1 from line 13
hdrln = split(readline(file))[5:end]
headr1 = hdrln[1:2:end]
units1 = hdrln[2:2:end]
readline(file) # skip line 14
# read header 2 from line 15
hdrln = split(readline(file))[4:end] # but don't parse it
headr2 = ["Range Gate", "Doppler", "Intensity", "Beta"]
units2 = ["none", "(m/s)", "(SNR + 1)", "(m-1 sr-1)"]
for _ in 16:17 # skip
    readline(file)
end

# now DATA
# beams could be rays or times
nbeams = round(Int, (nlines-17) / (1+ngates)) # = nrays*ntimes
data1 = zeros(nbeams, 5)
data2 = zeros(nbeams, ngates, 4)
# for User wind profiles beam <--> VAD ray
# for Stare beam <--> time
for ibeam = 1:nbeams
    # beam described by a batch of 1+ngates lines
    # Read the beam parameter line
    line = readline(file)
    data1[ibeam,:] .= parse.(Float64, split(line))
    # Read each gate in the beam
    for igate = 1:ngates
        line = readline(file)
        data2[ibeam, igate,:] .= parse.(Float64, split(line))
    end
end

# parse the variables
# beam
time      = data1[:,1] # decimal hours
azimuth   = data1[:,2] # degrees
elevangle = data1[:,3] # degrees
pitch     = data1[:,4]
roll      = data1[:,5]
# gate
height = (data2[1,:,1].+0.5) .* gatelength # center of gate

# dependent variables (beam, gate)
dopplervel = data2[:,:,2] # m/s
intensity  = data2[:,:,3] # SNR + 1
beta       = data2[:,:,4] # m-1 sr-1  backscatter?

5167×104 Matrix{Float64}:
  5.32249e-7  -3.95388e-7  9.86756e-5   …   4.40649e-6   4.76629e-6
  1.02162e-6  -1.1346e-6   0.00010017      -2.30728e-7   5.3197e-7
  5.53704e-7   2.16521e-7  9.76644e-5       4.67886e-7   2.85039e-6
  9.83356e-7  -1.67695e-6  0.000102148      2.32762e-6   2.12257e-6
  4.7144e-7   -3.60171e-6  9.78435e-5       7.15587e-7   2.54299e-6
  6.85693e-7  -1.18137e-7  9.93717e-5   …   2.64022e-6   2.48008e-6
  2.287e-7     2.73948e-7  0.000101613      2.92361e-6   2.73401e-6
  8.74478e-7  -5.67139e-7  0.000105571      3.54614e-6   3.46174e-6
  1.06741e-6  -2.09e-6     0.000104013      8.61251e-7   4.31232e-6
  9.51486e-7  -2.59142e-6  0.000101292      7.25583e-7   4.8531e-6
  ⋮                                     ⋱               
 -3.06658e-7  -6.72592e-7  0.000117056     -9.11084e-7   7.63524e-7
  5.14594e-8  -1.1646e-6   0.000117383      8.73456e-7   2.44613e-6
  1.83642e-7  -2.2522e-6   0.000119877  …  -1.45965e-6   3.61672e-6
  4.57835e-7  -2.75568e-6  0.000127

In [21]:

time, azimuth, elevangle, pitch, roll, height, dopplervel, intensity, beta = read_streamlinexr(file_path)

([6.0441, 6.04424444, 6.04438889, 6.04453611, 6.04468056, 6.044825, 6.04496944, 6.04511389, 6.04525833, 6.04540556  …  6.99861944, 6.99876389, 6.99890833, 6.99905278, 6.99919722, 6.99934167, 6.99948611, 6.99963056, 6.99977222, 6.99991667], [0.0, 360.0, 360.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 360.0, 0.0, 360.0], [90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0  …  90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0], [0.62, 0.82, 1.84, 1.44, -0.61, -0.91, -0.71, -1.73, -2.04, -2.34  …  -7.45, -7.34, -6.83, -7.45, -8.06, -9.59, -11.22, -10.1, -8.77, -4.89], [-1.62, -0.4, 2.66, 2.66, 4.18, 2.86, 3.06, 0.92, -1.11, -0.71  …  -1.01, -0.81, -0.4, 0.62, 1.43, 2.66, 4.39, 2.15, 0.72, 0.62], [15.0, 45.0, 75.0, 105.0, 135.0, 165.0, 195.0, 225.0, 255.0, 285.0  …  2835.0, 2865.0, 2895.0, 2925.0, 2955.0, 2985.0, 3015.0, 3045.0, 3075.0, 3105.0], [0.0618 -0.6978 … 1.999 -16.613; 0.0239 -0.7358 … 1.5812 1.2014; … ; -0.0141 -0.8118 … -0.7358 -0.